# [Double-Hard Debias: Tailoring Word Embeddings for Gender Bias Mitigation](https://arxiv.org/abs/2005.00965)

For more detailed explanations, please refer to the paper.

### Load original embeddings

In [ ]:
import codecs, os, json, operator, pickle, gensim
from random import shuffle
import numpy as np
from numpy import linalg as LA
import scipy

%load_ext autoreload
%autoreload 2

In [ ]:
def load_w2v(file_path):
    model =gensim.models.KeyedVectors.load_word2vec_format(file_path, binary=True)
    vocab = sorted([w for w in model.vocab], key=lambda w: model.vocab[w].index)
    w2i = {w: i for i, w in enumerate(vocab)}
    wv = [model[w] for w in vocab]
    wv = np.array(wv)
    print(len(vocab), wv.shape, len(w2i))
    
    return wv, w2i, vocab

file_path_wv = './data/GoogleNews-vectors-negative300.bin'
wv, w2i, vocab = load_w2v(file_path_wv)

### Remove Frequency Direction

In [ ]:
from sklearn.decomposition import PCA

# get main PCA components
def my_pca(wv):
    wv_mean = np.mean(np.array(wv), axis=0)
    wv_hat = np.zeros(wv.shape).astype(float)

    for i in range(len(wv)):
        wv_hat[i, :] = wv[i, :] - wv_mean

    main_pca = PCA()
    main_pca.fit(wv_hat)
    
    return main_pca

main_pca = my_pca(wv)
wv_mean = np.mean(np.array(wv), axis=0)

In [ ]:
def remove_frequency(wv, w2i, w2i_partial, vocab_partial, component_ids):
    
    D = []

    for i in component_ids:
        D.append(main_pca.components_[i])
    
    # get rid of frequency features
    wv_f = np.zeros((len(vocab_partial), wv.shape[1])).astype(float)
    
    for i, w in enumerate(vocab_partial):
        u = wv[w2i[w], :]
        sub = np.zeros(u.shape).astype(float)
        for d in D:
            sub += np.dot(np.dot(np.transpose(d), u), d)
        wv_f[w2i_partial[w], :] = wv[w2i[w], :] - sub - wv_mean
    
    print(wv_f.shape)
    return wv_f

In [ ]:
#Remove 8th component because authors found best performance 
#when the 8th component was removed.
component_id=7 
wv_f = remove_frequency(wv, w2i, w2i_partial = w2i, vocab_partial = vocab, component_ids = [component_id])

In [ ]:
with open("./data/w2v_frequency_removed.txt", "w") as outputFile:
    for i in range(len(vocab)):
        word = vocab[i]
        embedding = word + " " + " ".join([str(feature) for feature in wv_f[w2i[word],:]])
        outputFile.write(embedding + "\n")